Copyright (c) 2025 Mitsuru Ohno  
Use of this source code is governed by a BSD-3-style  
license that can be found in the LICENSE file.  

## 使用方法の要約
基本的な使用: RxnIVPsolv("sample_data.csv")でインスタンス化  
数値積分: get_ode_system()で必要なオブジェクトを取得  
エラーハンドリング: try-except文でエラーをキャッチ  
可視化: matplotlibで結果をプロット  
パラメータ解析: 異なる初期条件での比較  
これらの使用例は、現在のコードが正しく動作することを前提としています。もしエラーが発生した場合は、debug_ode_system()メソッドで詳細な情報を確認できます。  

## 反応式を記載したcsvファイルを指定する  

In [1]:
file_path = '../sample_data/sample_data1_2.csv'  # CSVファイルのパスを指定

In [2]:
# development phase
# Add the parent directory (one level up from the notebook's location) to the Python path
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '..'))

## 数値積分を実行する場合  

In [3]:
from dataclasses import dataclass, field
from typing import Optional

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from src.rxnfit.build_ode import RxnODEbuild

In [4]:
# 作成した微分方程式に関する情報を表示
def get_ode_info(rxn_ivp, debug_info=False):    
    print(f"number of species: {len(rxn_ivp.function_names)}")
    print(f"unique species: {rxn_ivp.function_names}")
    print(f"rate constant: {rxn_ivp.rate_consts_dict}")

    if debug_info is True:
        # デバッグ情報を確認
        print("\n=== debug info ===")
        debug_info = rxn_ivp.debug_ode_system()
        print(f"order of args: {debug_info['lambdify_args']}")
        print(f"system of ODE: {debug_info['ode_expressions']}")

In [5]:
@dataclass
class SolverConfig:
    rxn_ivp: RxnODEbuild
    y0: list              # 初期濃度（必須）
    t_span: tuple         # 時間範囲（必須）
    t_eval: Optional[np.ndarray] = field(default=None)  # 任意
    method: str = "RK45"  # 任意
    rtol: float = 1e-6    # 任意

In [ ]:
def solve_system(config: SolverConfig):

    # 数値積分に必要なオブジェクトを取得
    ode_construct = rxn_ivp_build.get_ode_system()
    (system_of_equations, sympy_symbol_dict, 
     ode_system, function_names, rate_consts_dict) = ode_construct
    
    # 微分方程式の右辺を定義
    def system_rhs(t, y):
        """ODEシステムの右辺を計算する関数"""
        rhs_odesys = []
        for i, species_name in enumerate(function_names):
            if species_name in ode_system:
                try:
                    rhs_odesys.append(ode_system[species_name](t, *y))
                except Exception as e:
                    print(f"Error in {species_name}: {e}")
                    rhs_odesys.append(0.0)
            else:
                rhs_odesys.append(0.0)
        return rhs_odesys
    
    # 数値積分を実行
    #print("\n=== 数値積分の実行 ===")
    solution = None
    try:
        solution = solve_ivp(
            system_rhs, 
            config.t_span, 
            config.y0, 
            t_eval=config.t_eval,
            method='RK45'
        )
        #print("数値積分が成功しました！")
    except Exception as e:
        print(f"数値積分でエラーが発生しました: {e}")
        print("デバッグ情報を確認してください。")
    return ode_construct, solution


In [ ]:
# 結果をプロット
def solution_plot(function_names, solution):
    print("\n=== 結果のプロット ===")
    plt.figure(figsize=(12, 8))
    
    for i, species_name in enumerate(function_names):
        plt.plot(solution.t, solution.y[i], label=species_name, linewidth=2)
    
    plt.xlabel('Time (s)', fontsize=12)
    plt.ylabel('Concentration', fontsize=12)
    plt.title('Chemical Reaction Kinetics - Sample Data', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 最終時刻での濃度を表示
    print("\n=== 最終時刻での濃度 ===")
    final_concentrations = {name: conc[-1] for name, conc in zip(function_names, solution.y)}
    for name, conc in final_concentrations.items():
        print(f"{name}: {conc:.6f}")
        


In [6]:
# 基本的な数値積分 -インスタンス化-
rxn_ivp_build = RxnODEbuild(file_path)

In [7]:
# 使い方例
rxn_ivp_build = RxnODEbuild(file_path)
get_ode_info(rxn_ivp_build, debug_info=True)

number of species: 8
unique species: ['AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']
rate constant: {'k1': 0.054, 'k2': k2, 'k3': 0.031}

=== debug info ===
Successfully created function for AcOEt with args: ['t', 'AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']
Successfully created function for OHa1 with args: ['t', 'AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']
Successfully created function for AcOa1 with args: ['t', 'AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']
Successfully created function for EtOH with args: ['t', 'AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']
Successfully created function for AcOiPr with args: ['t', 'AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']
Successfully created function for iPrOH with args: ['t', 'AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']
Successfully created function for EGOAc2 with args: ['t', 'AcOEt',

In [ ]:
# 初期値等の入力項目を渡す
config = SolverConfig(
    rxn_ivp=rxn_ivp_build, 
    y0=[1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    t_span=(0, 10)
)

In [12]:
rxn_ivp_build.rate_consts_dict['k1']+3

3.054

In [ ]:
intedrated_result = solve_system(config)
intedrated_result

In [ ]:
solution_plot(rxn_ivp_build.function_names,intedrated_result[1])

In [ ]:
solver = RxnODEsolver(SolverConfig(
    rxn_ivp=rxn_ivp_build,
    y0=[...],
    t_span=(0.0, 10.0),
    t_eval=np.linspace(0.0, 10.0, 200),
    method="RK45",
    rtol=1e-6,
))
solver.get_ode_info(debug_info=True)
ode_construct, sol = solver.solve_system()
solver.solution_plot()   # もしくは solver.solution_plot(sol)